In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
from tqdm import tqdm_notebook as tqdm

In [12]:
path = '../driver/chromedriver.exe'

In [58]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--profile-directory=Default')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--disable-plugins-discovery')
chrome_options.add_argument('--start-maximized')
# chrome_options.add_argument('headless')

browser = webdriver.Chrome(executable_path=path, options=chrome_options)
url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=data+science&locT=N&locId=96&jobType=&context=Jobs&sc.keyword=data+science&dropdown=0'
browser.get(url)

job_title = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Job Title, Keywords, or Company']")))
location = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Location']")))

job_title.clear()
location.clear()

job_title.send_keys('data science')
location.send_keys('Canada')

log_in = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

jobs = []
num_jobs=4
while len(jobs) < num_jobs:
        time.sleep(15)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            browser.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            browser.find_element_by_css_selector('[alt="Close"]').click() #clicking to the X.
            print(' x out worked')
        except NoSuchElementException:
            print(' x out failed')
            pass

        
        job_boxes = browser.find_elements_by_class_name("jl")
        for job_box in job_boxes:
#             print(job_box)
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
                
            job_box.click()
            time.sleep(1)
            
            collected_successfully = False
            while not collected_successfully:
                try:
                    company_name = browser.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = browser.find_element_by_xpath('.//div[@class="location"]').text
                    job_nu_naam = browser.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = browser.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text  # The role para on webpage
                    print(company_name, location, job_nu_naam)
                    collected_successfully = True
                except:
                    time.sleep(5)
                    
            try:
                salary_estimate = browser.find_element_by_xpath('.//span[@class="css-1uyte9r css-hca4ks e1wijj242"]').text  #class="css-1uyte9r css-hca4ks e1wijj242"
                print(salary_estimate)
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
                
                
                

 x out worked
Progress: 0/4
Spin Master Ltd
3.0 Toronto Senior Manager, Data Science
Progress: 0/4
Sun Life Financial
4.0 Toronto Director, Actuarial Analytics (Data Science)
Progress: 0/4
TwinThread Canada Inc. Bedford Sales Development Rep (SAAS startup; recent cold calling exp. req'd)
Progress: 0/4
Avanade
4.0 Toronto Data Engineer (Full Stack)
Progress: 0/4


ElementClickInterceptedException: Message: element click intercepted: Element <li class="jl react-job-listing gdGrid " data-brandviews="BRAND:n=jsearch-job-listing:eid=-1:jlid=3771817678" data-id="3771817678" data-adv-type="EMPLOYER" data-is-organic-job="false" data-ad-order-id="1044077" data-sgoc-id="1003" data-is-easy-apply="true" data-normalize-job-title="Business Development Manager" data-job-loc="Montreal" data-job-loc-id="2296722" data-job-loc-type="C" style="border-bottom:0">...</li> is not clickable at point (379, 670). Other element would receive the click: <p id="onetrust-policy-text">...</p>
  (Session info: chrome=87.0.4280.88)


In [2]:
def init_driver(path):    #path = '../driver/chromedriver.exe'
    # Initialize crome driver
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--profile-directory=Default')
    chrome_options.add_argument('--incognito')
    chrome_options.add_argument('--disable-pligins-discovery')
#     chrome_options.add_argument('--start-maximized')
    
    browser = webdriver.Chrome(executable_path=path, options=chrome_options)
    
    return browser

In [7]:
def search_jobs(job_name:str, job_location:str, num_jobs:int, sleep_time, verbose):
    
    # initialize webdriver
    browser = init_driver(path='../driver/chromedriver.exe') # driver =  browser  path = '../driver/chromedriver.exe'
    url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=data+science&locT=N&locId=96&jobType=&context=Jobs&sc.keyword=data+science&dropdown=0'
    browser.get(url)
    accept_cookies = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[id='onetrust-accept-btn-handler']"))).click()

    job_title = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Job Title, Keywords, or Company']")))
    location = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Location']")))
    
    job_title.clear()
    location.clear()
    time.sleep(5)  # wait for 5 seconds
    
    job_title.send_keys(job_name)
    location.send_keys(job_location)
    time.sleep(3)
    
    press_submit = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
#     time.sleep(sleep_time)
    time.sleep(3)
    jobs = []
#     tqdm().pandas()
    while len(jobs) < num_jobs:
###################CLOSE SIGN UP POP-UP#############################################        
        time.sleep(sleep_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            browser.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            browser.find_element_by_css_selector('[alt="Close"]').click() #clicking to the X.
            print(' x out worked')
        except NoSuchElementException:
            print(' x out failed')
            pass
####################################################################################        
        # Going through each job in this page
        job_boxes = browser.find_elements_by_class_name("jl")
        for job_box in job_boxes:
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
                
            job_box.click()
            time.sleep(1)
            
            collected_successfully = False
            while not collected_successfully:
                try:
                    company_name = browser.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = browser.find_element_by_xpath('.//div[@class="location"]').text
                    job_nu_naam = browser.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = browser.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text  # The role para on webpage
                    collected_successfully = True
                except:
                    time.sleep(5)
                    
            try:
                salary_estimate = browser.find_element_by_xpath('.//span[@class="css-1uyte9r css-hca4ks e1wijj242"]').text  # css-1uyte9r css-hca4ks e1wijj242
            except NoSuchElementException:
                salary_estimate = -1

            try:
                rating = browser.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1

            if verbose:
                print(f"Job Title: {job_nu_naam}")
                print(f"Salary Estimate: {salart_estimate}")
                print(f"Job Description: {job_description}")
                print(f"Rating: {rating}")
                print(f"Company Name: {company_name}")
                print(f"Location: {location}")

            #Going to the Company tab...    
#             try:
# #                 submit_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[data-tab-type="overview"]')))
# #                 submit_button.click()
#                 browser.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click() #and @data-test="tab" and @data-tab-type="overview"   //span[text()="Company"]//following-sibling::*
            try:
                headquarters = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
            except NoSuchElementException:
                headquarters = -1

            try:
                size = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text  #//span[@class="value"]
            except NoSuchElementException:
                size = -1

            try:
                founded = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
            except NoSuchElementException:
                founded = -1

            try:
                type_of_ownership = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
            except NoSuchElementException:
                type_of_ownership = -1

            try:
                industry = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
            except NoSuchElementException:
                industry = -1

            try:
                sector = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
            except NoSuchElementException:
                sector = -1

            try:
                revenue = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
            except NoSuchElementException:
                revenue = -1

            try:
                competitors = browser.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
            except NoSuchElementException:
                competitors = -1

            jobs.append({'Job Title':job_nu_naam,
                    "Salary Estimate" : salary_estimate,
                    "Job Description" : job_description,
                    "Rating" : rating,
                    "Company Name" : company_name,
                    "Location" : location,
                    "Headquarters" : headquarters,
                    "Size" : size,
                    "Founded" : founded,
                    "Type of ownership" : type_of_ownership,
                    "Industry" : industry,
                    "Sector" : sector,
                    "Revenue" : revenue,
                    "Competitors" : competitors})

#             except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
#                 pass

            if verbose:
                print(f"Headquaters: {headquarters}")
                print(f"Size: {size}")
                print(f"Founded: {founded}")
                print(f"Type of Ownership: {type_of_ownership}")
                print(f"Industry: {industry}")
                print(f"Sector:{sector}")
                print(f"Revenue: {revenue}")
                print(f"Competitors: {competitors}")
                print("#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$")

            
            
        try:
            browser.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print(f"Scraping terminated before reaching target of jobs. Needed {num_jobs} , got {len(jobs)}")

    return pd.DataFrame(jobs)


In [8]:
df = search_jobs(job_name='data scientist/data science', job_location='London, England', num_jobs=1000, sleep_time=10, verbose=False)
df

 x out worked
Progress: 0/1000
Progress: 1/1000
Progress: 2/1000
Progress: 3/1000


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=87.0.4280.88)
